In [ ]:
%load_ext autoreload
%autoreload 2

# Utils

In [ ]:
#|default_exp utils

In [ ]:
#|export
import os
import openai
from dotenv import load_dotenv
import requests
import fastcore.all as fc

In [ ]:
load_dotenv()

True

In [ ]:
#|export
def fw(): return openai.OpenAI(base_url="https://api.fireworks.ai/inference/v1", api_key=os.environ['FIREWORKS_API_KEY'])

In [ ]:
fw()

<openai.OpenAI>

In [ ]:
#|export
class Reranker:
    def __init__(self, model='accounts/fireworks/models/qwen3-reranker-8b', base_url="https://api.fireworks.ai/inference/v1", api_key=None):
        if api_key is None: api_key = os.environ['FIREWORKS_API_KEY']
        fc.store_attr()
        
    def rank(self, query, docs, n=10, return_documents=False, task="Given a web search query, retrieve relevant passages that answer the query"):
        ans = fc.L.range(len(docs))
        try:
            res = requests.post(f"{self.base_url}/rerank",
                json={"model": self.model, "query": query, "documents": list(docs), "top_n": n, "return_documents": False},
                headers={"Authorization": f"Bearer {self.api_key}", "Content-Type": "application/json"}
            )
            return fc.L(res.json()['data']).attrgot('index')
        except Exception as e:
            print(e)
        return ans

In [ ]:
ranker = Reranker('accounts/fireworks/models/qwen3-reranker-8b')
query = "India's performance in international sports and current economic outlook"

docs = ['India clinches 6th Under-19 World Cup title with a dominant 100-run victory over England', 
        "RBI keeps repo rate unchanged at 5.25% while maintaining 'neutral' policy stance",
        'Vaibhav Sooryavanshi smashes a commanding 175 to lead India’s victory on the grand stage',
        'Olympic flame arrives in Milan for 2026 Winter Games featuring star-studded guest list',
        'Singapore woman fined ₹2 lakh for "one bad habit"—here is why']

In [ ]:
Reranker().rank(query, docs)

(#5) [0,2,1,3,4]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()